import os
import json
import pandas as pd
import re
from tabulate import tabulate
# File paths
csv_file = "C:\\Users\\anish\\OneDrive\\Documents\\GitHub\\mainfolder\\sales_data.csv"
json_folder = "C:\\Users\\anish\\OneDrive\\Documents\\GitHub\\mainfolder\\product_details"
txt_folder = "C:\\Users\\anish\\OneDrive\\Documents\\GitHub\\mainfolder\\product_descriptions"

'''def load_sales_data(csv_file):
    """Load sales data from a CSV file and return as a dictionary."""
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file)
        return {row["Product_SKU"]: row.iloc[1:].tolist() for _, row in df.iterrows()}
    else:
        return {}'''


def load_sales_data(csv_file):
    """Load sales data from a CSV file and return as a dictionary."""
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file)

        # Ensure 'Product_SKU' column exists
        if "Product_SKU" not in df.columns:
            print("❌ Error: 'Product_SKU' column not found in CSV!")
            return {}

        # Fill missing values with 0 and ensure numerical data
        df.fillna(0, inplace=True)

        # Convert all day columns to integers
        for col in df.columns[1:]:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

        # Return data as a dictionary
        return df

    else:
        print("❌ Error: CSV file not found!")
        return pd.DataFrame()  # Return empty DataFrame if file doesn't exist

# Load sales data
df_sales = load_sales_data(csv_file)

if not df_sales.empty:
    print("\n📊 SALES DATA TABLE:\n")

    # Display data in tabular format without using tabulate
    print(df_sales.to_string(index=False))  # Prints DataFrame in a structured table-like format
else:
    print("⚠ No sales data available.")


def load_product_details(json_folder):
    """Load product details from JSON files."""
    product_details = {}
    if os.path.exists(json_folder):
        for filename in os.listdir(json_folder):
            if filename.startswith("details_") and filename.endswith('.json'):
                sku = filename.replace("details_", "").replace('.json', '')
                with open(os.path.join(json_folder, filename), 'r', encoding='utf-8') as file:
                    product_details[sku] = json.load(file)
    return product_details

def load_product_descriptions(txt_folder):
    """Load product descriptions from text files."""
    product_descriptions = {}
    if os.path.exists(txt_folder):
        for filename in os.listdir(txt_folder):
            if filename.startswith("description_") and filename.endswith('.txt'):
                sku = filename.replace("description_", "").replace('.txt', '')
                with open(os.path.join(txt_folder, filename), 'r', encoding='utf-8') as file:
                    product_descriptions[sku] = file.read().strip()
    return product_descriptions



def load_data():
    """Load all required data files into structured formats and display in a formatted way."""
    sales_data = load_sales_data(csv_file)
    product_details = load_product_details(json_folder)
    product_descriptions = load_product_descriptions(txt_folder)

    # Convert sales_data dictionary to DataFrame for better readability
    df_sales = pd.DataFrame.from_dict(sales_data, orient='index', columns=[f"Day{i}" for i in range(1, 15)])
    df_sales.insert(0, "Product_SKU", df_sales.index)  # Insert SKU column at the beginning

    # Display sales data in tabular format
    print("\n📊 SALES DATA TABLE:\n")
    print(tabulate(df_sales, headers="keys", tablefmt="grid", showindex=False))

    # Display product details in readable JSON format
    print("\n📦 PRODUCT DETAILS:\n")
    for sku, details in product_details.items():
        print(f"SKU: {sku}")
        print(json.dumps(details, indent=4))  # Pretty-print JSON
        print("-" * 50)

    # Display product descriptions
    print("\n📝 PRODUCT DESCRIPTIONS:\n")
    for sku, description in product_descriptions.items():
        print(f"SKU: {sku}")
        print(description)
        print("-" * 50)

    return sales_data, product_details, product_descriptions

# Load initial data
sales_data, product_details, product_descriptions = load_data()
print("✅ Data Loaded Successfully")

def validate_sku(sku):
    """Validate SKU: It should be exactly 13 characters, alphanumeric, and uppercase."""
    return bool(re.fullmatch(r"[A-Z0-9]{13}", sku))

def validate_sales_data(data):
    """Validate sales data: It should be 14 whole numbers separated by spaces."""
    numbers = data.split()
    return len(numbers) == 14 and all(num.isdigit() for num in numbers)

def update_sales_data():
    """Update or add sales data for a product SKU."""
    global sales_data
    
    sku = input("Enter Product Sales SKU: ").strip()
    if not validate_sku(sku):
        print("❌ Invalid SKU! It should be exactly 13 characters, alphanumeric, and uppercase.")
        return
    
    if sku in sales_data:
        choice = input("This SKU ID is found. Do you want to update the data? (yes/no): ").strip().lower()
        if choice != 'yes':
            return
        day = input("Enter specific day to update (1-14) or press Enter to update all days: ")
        if day:
            day = int(day)
            sales = int(input(f"Enter sales data for Day {day}: "))
            sales_data[sku][day - 1] = sales
        else:
            sales_data_input = input("Enter sales data for all 14 days (separated by spaces): ")
            if not validate_sales_data(sales_data_input):
                print("❌ Invalid sales data! It should contain exactly 14 whole numbers separated by spaces.")
                return
            sales_data[sku] = list(map(int, sales_data_input.split()))
    else:
        choice = input("This SKU ID is not found. Do you want to add new data? (yes/no): ").strip().lower()
        if choice != 'yes':
            return
        sales_data_input = input("Enter sales data for all 14 days (separated by spaces): ")
        if not validate_sales_data(sales_data_input):
            print("❌ Invalid sales data! It should contain exactly 14 whole numbers separated by spaces.")
            return
        sales_data[sku] = list(map(int, sales_data_input.split()))

    print("✅ Sales data updated successfully!")

def update_product_details():
    """Update or add product details in JSON format."""
    sku = input("Enter Product Detail SKU: ").strip()
    if not validate_sku(sku):
        print("❌ Invalid SKU!")
        return
    
    details = {
        "product_name": input("Enter Product Name: "),
        "brand": input("Enter Brand: "),
        "model": input("Enter Model: "),
        "specifications": input("Enter Specifications: "),
        "price": input("Enter Price: "),
        "availability": input("Enter Availability: ")
    }
    
    product_details[sku] = details
    print("✅ Product details updated successfully!")

def update_product_description():
    """Update or add product description in a text file."""
    sku = input("Enter Product Description SKU: ").strip()
    if not validate_sku(sku):
        print("❌ Invalid SKU!")
        return
    
    product_descriptions[sku] = input("Enter Product Description: ")
    print("✅ Product description updated successfully!")


def update():
    """Calls all update functions for sales, product details, and descriptions."""
    update_sales_data()
    update_product_details()
    update_product_description()

# Call update function when needed
print("🔄 Ready to update data")

def dump_data():
    """Dump updated data into files, ensuring the correct folder structure."""
    
    # Define main folder path
    main_folder = "C:\\Users\\anish\\OneDrive\\Documents\\GitHub\\mainfolder"

    # Ensure main folder exists
    os.makedirs(main_folder, exist_ok=True)
    
    # Convert sales_data dictionary back to DataFrame before saving
    df_sales = pd.DataFrame.from_dict(sales_data, orient='index', columns=[f"Day{i}" for i in range(1, 15)])
    df_sales.insert(0, "Product_SKU", df_sales.index)
    df_sales.to_csv(os.path.join(main_folder, "sales_data.csv"), index=False)
    print("✅ Sales data successfully written to CSV.")

    # Ensure subfolders exist
    product_details_folder = os.path.join(main_folder, "product_details")
    product_descriptions_folder = os.path.join(main_folder, "product_descriptions")
    os.makedirs(product_details_folder, exist_ok=True)
    os.makedirs(product_descriptions_folder, exist_ok=True)

    # Save product details
    for sku, details in product_details.items():
        with open(os.path.join(product_details_folder, f"details_{sku}.json"), "w", encoding="utf-8") as json_file:
            json.dump(details, json_file, indent=4)
    print("✅ Product details successfully written to JSON files.")

    # Save product descriptions
    for sku, description in product_descriptions.items():
        with open(os.path.join(product_descriptions_folder, f"description_{sku}.txt"), "w", encoding="utf-8") as txt_file:
            txt_file.write(description)
    print("✅ Product descriptions successfully written to text files.")

# Call dump_data() when needed
print("📂 Ready to dump data")


def fmain():
    load_data()
print('✅ Data Loaded Successfully')
up = input('Do you want to update the data :- (yes/no)').strip().lower()
if up == 'yes':
    update()
    print('update successfully')
    dump_data()
    load_data()
else:
    print('thank you !!!!!!')

load_sales_data(csv_file)

import os
import json
import pandas as pd
import re
from tabulate import tabulate

# File paths
csv_file = "C:\\Users\\anish\\OneDrive\\Documents\\GitHub\\mainfolder\\sales_data.csv"
json_folder = "C:\\Users\\anish\\OneDrive\\Documents\\GitHub\\mainfolder\\product_details"
txt_folder = "C:\\Users\\anish\\OneDrive\\Documents\\GitHub\\mainfolder\\product_descriptions"

def load_sales_data(csv_file):
    """Load sales data from a CSV file and return as a DataFrame."""
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file)

        # Ensure 'Product_SKU' column exists
        if "Product_SKU" not in df.columns:
            print("❌ Error: 'Product_SKU' column not found in CSV!")
            return pd.DataFrame()  # Return empty DataFrame

        # Fill missing values with 0 and ensure numerical data
        df.fillna(0, inplace=True)

        # Convert all day columns to integers
        for col in df.columns[1:]:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

        return df
    else:
        print("❌ Error: CSV file not found!")
        return pd.DataFrame()  # Return empty DataFrame if file doesn't exist

def load_product_details(json_folder):
    """Load product details from JSON files."""
    product_details = {}
    if os.path.exists(json_folder):
        for filename in os.listdir(json_folder):
            if filename.startswith("details_") and filename.endswith('.json'):
                sku = filename.replace("details_", "").replace('.json', '')
                with open(os.path.join(json_folder, filename), 'r', encoding='utf-8') as file:
                    product_details[sku] = json.load(file)
    return product_details

def load_product_descriptions(txt_folder):
    """Load product descriptions from text files."""
    product_descriptions = {}
    if os.path.exists(txt_folder):
        for filename in os.listdir(txt_folder):
            if filename.startswith("description_") and filename.endswith('.txt'):
                sku = filename.replace("description_", "").replace('.txt', '')
                with open(os.path.join(txt_folder, filename), 'r', encoding='utf-8') as file:
                    product_descriptions[sku] = file.read().strip()
    return product_descriptions

def load_data():
    """Load all required data files and display in a formatted way."""
    sales_data = load_sales_data(csv_file)  # This is now a DataFrame
    product_details = load_product_details(json_folder)
    product_descriptions = load_product_descriptions(txt_folder)

    # Display sales data in tabular format
    if not sales_data.empty:
        print("\n📊 SALES DATA TABLE:\n")
        print(tabulate(sales_data, headers="keys", tablefmt="grid", showindex=False))
    else:
        print("⚠ No sales data available.")

    # Display product details in readable JSON format
    print("\n📦 PRODUCT DETAILS:\n")
    for sku, details in product_details.items():
        print(f"SKU: {sku}")
        print(json.dumps(details, indent=4))  # Pretty-print JSON
        print("-" * 50)

    # Display product descriptions
    print("\n📝 PRODUCT DESCRIPTIONS:\n")
    for sku, description in product_descriptions.items():
        print(f"SKU: {sku}")
        print(description)
        print("-" * 50)

    return sales_data, product_details, product_descriptions  # Return the DataFrame directly

# Load initial data
sales_data, product_details, product_descriptions = load_data()
print("✅ Data Loaded Successfully")

def validate_sku(sku):
    """Validate SKU: It should be exactly 13 characters, alphanumeric, and uppercase."""
    return bool(re.fullmatch(r"[A-Z0-9]{13}", sku))

def update_sales_data():
    """Update or add sales data for a product SKU."""
    global sales_data

    sku = input("Enter Product Sales SKU: ").strip()
    if not validate_sku(sku):
        print("❌ Invalid SKU! It should be exactly 13 characters, alphanumeric, and uppercase.")
        return

    if sku in sales_data["Product_SKU"].values:
        choice = input("This SKU ID is found. Do you want to update the data? (yes/no): ").strip().lower()
        if choice != 'yes':
            return
        day = input("Enter specific day to update (1-14) or press Enter to update all days: ")
        if day:
            day = int(day)
            sales = int(input(f"Enter sales data for Day {day}: "))
            sales_data.loc[sales_data["Product_SKU"] == sku, f"Day{day}"] = sales
        else:
            sales_data_input = input("Enter sales data for all 14 days (separated by spaces): ")
            sales_values = list(map(int, sales_data_input.split()))
            for i, val in enumerate(sales_values):
                sales_data.loc[sales_data["Product_SKU"] == sku, f"Day{i+1}"] = val
    else:
        choice = input("This SKU ID is not found. Do you want to add new data? (yes/no): ").strip().lower()
        if choice != 'yes':
            return
        sales_data_input = input("Enter sales data for all 14 days (separated by spaces): ")
        sales_values = list(map(int, sales_data_input.split()))
        new_row = {"Product_SKU": sku, **{f"Day{i+1}": sales_values[i] for i in range(14)}}
        sales_data = sales_data.append(new_row, ignore_index=True)

    print("✅ Sales data updated successfully!")

def dump_data():
    """Dump updated data into files, ensuring the correct folder structure."""
    main_folder = "C:\\Users\\anish\\OneDrive\\Documents\\GitHub\\mainfolder"

    os.makedirs(main_folder, exist_ok=True)

    # Save sales data
    sales_data.to_csv(os.path.join(main_folder, "sales_data.csv"), index=False)
    print("✅ Sales data successfully written to CSV.")

    product_details_folder = os.path.join(main_folder, "product_details")
    product_descriptions_folder = os.path.join(main_folder, "product_descriptions")

    os.makedirs(product_details_folder, exist_ok=True)
    os.makedirs(product_descriptions_folder, exist_ok=True)

    for sku, details in product_details.items():
        with open(os.path.join(product_details_folder, f"details_{sku}.json"), "w", encoding="utf-8") as json_file:
            json.dump(details, json_file, indent=4)

    print("✅ Product details successfully written to JSON files.")

    for sku, description in product_descriptions.items():
        with open(os.path.join(product_descriptions_folder, f"description_{sku}.txt"), "w", encoding="utf-8") as txt_file:
            txt_file.write(description)

    print("✅ Product descriptions successfully written to text files.")

print("📂 Ready to dump data")


Enter Product Sales SKU:  123456789123H
This SKU ID is found. Do you want to update the data? (yes/no):  yes
Enter specific day to update (1-14) or press Enter to update all days:  1
Enter sales data for Day 1:  200


✅ Sales data updated successfully!


Enter Product Detail SKU:  123456789123H
Enter Product Name:  gaming laptop
Enter Brand:  lenevo
Enter Model:  v15
Enter Specifications:  athlon gold
Enter Price:  37000
Enter Availability:  in stock 


✅ Product details updated successfully!


Enter Product Description SKU:  123456789123H
Enter Product Description:  amazing laptop with power house performance


✅ Product description updated successfully!


In [ ]:
import os
import json
import pandas as pd
import re
from tabulate import tabulate

# File paths
main_folder = "C:\\Users\\anish\\OneDrive\\Documents\\GitHub\\mainfolder"
csv_file = os.path.join(main_folder, "sales_data.csv")
json_folder = os.path.join(main_folder, "product_details")
txt_folder = os.path.join(main_folder, "product_descriptions")

def load_sales_data(csv_file):
    """Load sales data from a CSV file and return as a DataFrame."""
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file)
        if "Product_SKU" not in df.columns:
            print("❌ Error: 'Product_SKU' column not found in CSV!")
            return pd.DataFrame()
        df.fillna(0, inplace=True)
        for col in df.columns[1:]:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
        return df
    else:
        print("❌ Error: CSV file not found!")
        return pd.DataFrame()

def load_product_details(json_folder):
    """Load product details from JSON files."""
    product_details = {}
    if os.path.exists(json_folder):
        for filename in os.listdir(json_folder):
            if filename.startswith("details_") and filename.endswith('.json'):
                sku = filename.replace("details_", "").replace('.json', '')
                with open(os.path.join(json_folder, filename), 'r', encoding='utf-8') as file:
                    product_details[sku] = json.load(file)
    return product_details

def load_product_descriptions(txt_folder):
    """Load product descriptions from text files."""
    product_descriptions = {}
    if os.path.exists(txt_folder):
        for filename in os.listdir(txt_folder):
            if filename.startswith("description_") and filename.endswith('.txt'):
                sku = filename.replace("description_", "").replace('.txt', '')
                with open(os.path.join(txt_folder, filename), 'r', encoding='utf-8') as file:
                    product_descriptions[sku] = file.read().strip()
    return product_descriptions

def validate_sku(sku):
    """Validate SKU: It should be exactly 13 characters, alphanumeric, and uppercase."""
    return bool(re.fullmatch(r"[A-Z0-9]{13}", sku))

def update_sales_data(sku, sales_values):
    """Update or add sales data for a product SKU."""
    global sales_data
    if sku in sales_data["Product_SKU"].values:
        for i, val in enumerate(sales_values):
            sales_data.loc[sales_data["Product_SKU"] == sku, f"Day{i+1}"] = val
    else:
        new_row = {"Product_SKU": sku, **{f"Day{i+1}": sales_values[i] for i in range(14)}}
        sales_data = sales_data.append(new_row, ignore_index=True)
    print("✅ Sales data updated successfully!")


def update_product_details(sku, name, brand, model, specs, price, availability):
    """Update or add product details."""
    product_details[sku] = {
        "Product Name": name,
        "Brand": brand,
        "Model": model,
        "Specifications": specs,
        "Price": price,
        "Availability": availability
    }
    print("✅ Product details updated successfully!")

def update_product_description(sku, description):
    """Update or add product description."""
    product_descriptions[sku] = description
    print("✅ Product description updated successfully!")

def update():
    """Collect input and update all records."""
    sku = input("Enter Product SKU: ").strip()
    if not validate_sku(sku):
        print("❌ Invalid SKU! It should be exactly 13 characters, alphanumeric, and uppercase.")
        return
    
    sales_data_input = input("Enter sales data for all 14 days (separated by spaces): ")
    sales_values = list(map(int, sales_data_input.split()))
    if len(sales_values) != 14:
        print("❌ Sales data must contain exactly 14 values.")
        return
    
    name = input("Enter Product Name: ")
    brand = input("Enter Brand: ")
    model = input("Enter Model: ")
    specs = input("Enter Specifications: ")
    price = input("Enter Price: ")
    availability = input("Enter Availability: ")
    description = input("Enter Product Description: ")
    
    update_sales_data(sku, sales_values)
    update_product_details(sku, name, brand, model, specs, price, availability)
    update_product_description(sku, description)
    print("✅ All product details updated successfully!")

def dump_data():
    """Dump updated data into files."""
    os.makedirs(main_folder, exist_ok=True)
    sales_data.to_csv(csv_file, index=False)
    print("✅ Sales data successfully written to CSV.")
    
    os.makedirs(json_folder, exist_ok=True)
    os.makedirs(txt_folder, exist_ok=True)
    
    for sku, details in product_details.items():
        with open(os.path.join(json_folder, f"details_{sku}.json"), "w", encoding="utf-8") as json_file:
            json.dump(details, json_file, indent=4)
    print("✅ Product details successfully written to JSON files.")
    
    for sku, description in product_descriptions.items():
        with open(os.path.join(txt_folder, f"description_{sku}.txt"), "w", encoding="utf-8") as txt_file:
            txt_file.write(description)
    print("✅ Product descriptions successfully written to text files.")

# Load initial data
sales_data = load_sales_data(csv_file)
product_details = load_product_details(json_folder)
product_descriptions = load_product_descriptions(txt_folder)
print("✅ Data Loaded Successfully")

# Example of updating with given data
update_sales_data("CMWKCILOP27KF", [8, 14, 16, 7, 15, 21, 14, 16, 32, 29, 26, 30, 25, 22])
update_product_details("CMWKCILOP27KF", "Pokemon Card", "GameFreak", "ScarletViolet151", "Genuine, TCG, English", "$1.99", "In stock")
update_product_description("CMWKCILOP27KF", "Original Pokemon TCG Pikachu card")

dump_data()


✅ Data Loaded Successfully


In [ ]:
def fmain():
    load_data()
print('✅ Data Loaded Successfully')
up = input('Do you want to update the data :- (yes/no)').strip().lower()
if up == 'yes':
    update()
    print('update successfully')
    dump_data()
    load_data()
else:
    print('thank you !!!!!!')

✅ Data Loaded Successfully
